## Construindo uma CNN para classificar imagens no Dataset CIFAR-10

Trabalharemos com o Dataset CIFAR-10. Este é um conjunto de dados bem conhecido para classificação de imagens, que consiste em 60.000 imagens coloridas de 32x32 pixels divididas em 10 classes, com 6.000 imagens por classe. Há 50.000 imagens de treinamento e 10.000 imagens de teste.

As 10 classes são:

<ol start="0">
<li> avião
<li> automóvel
<li> pássaro
<li> gato
<li> veado
<li> cachorro
<li> sapo
<li> cavalo
<li> navio
<li> caminhão
</ol>

Para detalhes sobre o CIFAR-10, consulte:
https://www.cs.toronto.edu/~kriz/cifar.html

Para uma compilação de resultados de desempenho publicados no CIFAR-10, consulte:
http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

---

### Construindo Redes Neurais Convolucionais

Neste exercício, construiremos e treinaremos nossas primeiras redes neurais convolucionais. Na primeira parte, percorreremos as diferentes camadas e como elas são configuradas. Na segunda parte, você construirá seu próprio modelo, o treinará e comparará o desempenho.

In [ ]:
# Preliminares

from __future__ import absolute_import, division, print_function  # Compatibilidade Python 2/3

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Importar objetos do Keras para Deep Learning

import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Os dados, embaralhados e divididos entre conjuntos de treino e teste:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('formato x_train:', x_train.shape)
print(x_train.shape[0], 'amostras de treino')
print(x_test.shape[0], 'amostras de teste')

In [ ]:
# Cada imagem é um array numpy 32 x 32 x 3
x_train[444].shape

In [ ]:
# Vamos observar uma das imagens

print(y_train[444])
plt.imshow(x_train[444]);

In [ ]:
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# agora, em vez de classes descritas por um inteiro entre 0-9, temos um vetor com um 1 na 9ª posição (indexação do Python)
y_train[444]

In [ ]:
# Como antes, vamos fazer tudo float e escalar
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Camadas do Keras para CNNs
- Anteriormente, construímos Redes Neurais usando principalmente as camadas Dense, Activation e Dropout.

- Aqui descreveremos como usar algumas das camadas específicas para CNN fornecidas pelo Keras

### Conv2D

```python
keras.layers.convolutional.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
```

Alguns parâmetros explicados:
- `filters`: o número de filtros usados por localização. Em outras palavras, a profundidade da saída.
- `kernel_size`: uma tupla (x,y) fornecendo a altura e largura do kernel a ser usado
- `strides`: uma tupla (x,y) fornecendo o passo em cada dimensão. O padrão é `(1,1)`
- `input_shape`: necessário apenas para a primeira camada

Note que o tamanho da saída será determinado pelo kernel_size e strides

### MaxPooling2D
`keras.layers.pooling.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)`

- `pool_size`: o tamanho (x,y) da grade a ser agrupada.
- `strides`: Assumido como sendo o `pool_size` a menos que especificado de outra forma

### Flatten
Transforma sua entrada em um vetor unidimensional (por instância). Geralmente usado ao fazer a transição entre camadas convolucionais e camadas totalmente conectadas.

---

## Primeira CNN
Abaixo construiremos nossa primeira CNN. Para fins de demonstração (para que treine rapidamente), ela não é muito profunda e tem relativamente poucos parâmetros. Usamos strides de 2 nas duas primeiras camadas convolucionais, o que reduz rapidamente as dimensões da saída. Após uma camada MaxPooling, fazemos o flatten e então temos uma única camada totalmente conectada antes de nossa camada de classificação final.

In [ ]:
# Vamos construir uma CNN usando as capacidades Sequential do Keras

model_1 = Sequential()


# Convolução 5x5 com stride 2x2 e 32 filtros
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

# Outra convolução 5x5 com stride 2x2 e 32 filtros
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

# Max pooling 2x2 reduz para 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

# Flatten transforma 3x3x32 em 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

Ainda temos 181K parâmetros, mesmo sendo um modelo "pequeno".

In [ ]:
batch_size = 32

# inicializar otimizador RMSprop
opt = RMSprop(learning_rate=0.0005, decay=1e-6)

# Vamos treinar o modelo usando RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15,
              validation_data=(x_test, y_test),
              shuffle=True)

### Exercício
Nosso modelo anterior tinha a estrutura:

Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Classificação Final

(com funções de ativação e dropouts apropriados)

1. Construa um modelo mais complicado com o seguinte padrão:
- Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Classificação Final

- Use strides de 1 para todas as camadas convolucionais.

2. Quantos parâmetros seu modelo tem? Como isso se compara ao modelo anterior?

3. Treine-o por 5 épocas. O que você observa sobre o tempo de treinamento, perda e números de acurácia (tanto nos conjuntos de treinamento quanto de validação)?

5. Experimente diferentes estruturas e tempos de execução, e veja quão preciso seu modelo pode ser.

In [ ]:
# Forneça sua solução aqui
# Crie model_2 conforme mencionado no exercício